# Decision Tree exercise
You should use decision tree to classify. 

Design your DecisionTree. Do binary classification or multiclass classification (selected by yourself)

In [248]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn import tree

%matplotlib inline

## Kaggle in-class Competetion
请先前往 Kaggle 下载本次比赛的数据集

比赛页面：https://inclass.kaggle.com/c/hdu-cama/leaderboard

本次比赛可使用的 Package: Pandas, Numpy 以及系统内置库如 math 等

完成下面代码后，使用 predict 函数对 test.csv 中的数据做出预测并将结果保存至一个 .csv 文件，然后 submit 至 Kaggle，可参考示例文件 sample.csv

__请务必仔细阅读 Kaggle 页面的各项信息__

__请务必仔细阅读 Kaggle 页面的各项信息__

__请务必仔细阅读 Kaggle 页面的各项信息__

## 请务必仔细阅读文件 “ID3 Algorithm for Decision Trees.pdf”
## 请务必仔细阅读文件 “ID3 Algorithm for Decision Trees.pdf”
## 请务必仔细阅读文件 “ID3 Algorithm for Decision Trees.pdf”
### Calculate Shannon Entropy

熵是对不确定性的测量，熵越高，代表信息量越高，这里你需要使用熵来选择作为节点的特征。（选择能够最小化两边熵的特征）

$$Entropy(S) = - P_+ \log_2{P_+} - P_- \log_2{P_-}$$

### Calculate Information Gain
$$Gain(S, A) = Entropy(S) - \sum_{v\in Values(A)}{\frac{|S_v|}{|S|}Entropy(S_v)}$$

In [249]:
# Read data from train.csv and y_train.csv
train_feature = pd.read_csv('train.csv')
train_label = pd.read_csv('y_train.csv')

train_dataSet = pd.merge(train_feature, tarin_label, on = 'ID')

featureNames = train_dataSet.columns.tolist()[1:10]

dataSet_inArray = train_dataSet.values
dataSet_inArray_noSerial = dataSet_inArray[:,1:]
dataSet_totalSplit = dataSet_inArray_noSerial.tolist()

In [250]:
def calculateShannonEntropy(dataSet):
    # Todo 1: calculate the entropy given a dataset
   
    numEntries = len(dataSet) # There are n rows inside
    labelCounts = {} # Create dictionary for classification

    for featureVector in dataSet:
        
    	currentLabel = featureVector[-1] # Get the last-row data
    	if currentLabel not in labelCounts.keys():
    		labelCounts[currentLabel] = 0
    	labelCounts[currentLabel] += 1

    total_entropy = 0.0
    for key in labelCounts:
    	proportion_k = float(labelCounts[key]) / numEntries
    	total_entropy -= (proportion_k * log(proportion_k, 2))

    return total_entropy

calculateShannonEntropy(dataSet_totalSplit)

0.9278532379384186

In [251]:
def choose_best_feature_to_split(dataSet):
    # Todo 2: return the best feature based on the maximum number of information gain
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calculateShannonEntropy(dataSet)
    bestInfoGain = 0
    best_feature = 0

    for i in range(numFeatures):
    	featureList = [number[i] for number in dataSet] # enum for one attribute
    	uniqualValues = set(featureList) # no-relace attribute
    	newEntropy = 0

    	for value in uniqualValues:
    		sub_dataset = split_dataset(dataSet, i, value)
    		proportion_k = len(sub_dataset) / float(len(dataSet))
    		newEntropy += proportion_k * calculateShannonEntropy(sub_dataset) # sum(ShannonEntropy)
    	infoGain = baseEntropy - newEntropy # infoGain

    	# bestInfoGain
    	if (infoGain > bestInfoGain):
    		bestInfoGain = infoGain
    		best_feature = i

    return best_feature
choose_best_feature_to_split(dataSet_totalSplit)

4

In [252]:
def split_dataset(dataSet, axis, value):
    # Todo 3: Split the dataset via current selected feature and it's value
    # For example, when current_feature is TLS(top-left-square), and the value is 'o', 
    # the task is that return the subdataset in which all "TLS" is equal to 'o'
    sub_dataset = []

    for featureVector in dataSet:
    	if featureVector[axis] == value:
    		reduceFeatureVector = featureVector[ :axis]
    		reduceFeatureVector.extend(featureVector[axis+1: ])  
    		sub_dataset.append(reduceFeatureVector)

    return sub_dataset

In [253]:
import operator
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():classCount[vote] = 0
        classCount[vote]+=1
    sortedClassCount=sorted(classCount.items(),key = operator.itemgetter(1),reverse = True)
    return sortedClassCount[0][0]

def create_decision_tree(dataSet, featureNames):
    # Todo 4: Create a decision tree by recursion
    #
    # Tips: Set appropriate boundary conditions; 
    #       think about the values one by one; 
    #       Use the three functions defined before.
    
    classList = [example[0] for example in dataSet]
    #类别相同，停止划分
    if classList.count(classList[-1]) == len(classList):
        return classList[0]
    #长度为1，返回出现次数最多的类别
    if len(classList[0]) == 1:
        return majorityCnt(classList)

    best_feature = choose_best_feature_to_split(dataSet) #返回分类的特征序号
    bestFeatureName = featureNames[best_feature] #该特征的label
    decision_tree = {bestFeatureName: { } }
    del(featureNames[best_feature]) #从labels的list中删除该label
    
    featureValues = [example[best_feature] for example in dataSet]
    uniqualValues = set(featureValues)
    for value in uniqualValues:
    	subFeatureNames = featureNames[ : ] #子集合

    	#构建数据的子集合，并进行递归
    	decision_tree[bestFeatureName][value] = create_decision_tree(split_dataset(dataSet, best_feature, value), subFeatureNames)
    
    return decision_tree

myDecisionTree = create_decision_tree(dataSet_totalSplit, featureNames)
print(myDecisionTree)

x


In [257]:
# Get test_dataset from test.csv
test_dataset = pd.read_csv('test.csv')

# Todo 5

def classify(inputTree, featureNames, testVector):
    
    firstStr = inputTree.keys()[0] #获取树的第一个特征属性
    secondDict = inputTree[firstStr] #树的分支，子集合Dict
    featureIndex = featureNames.index(firstStr) #获取决策树第一层在featLables中的位置
    for key in secondDict.keys():
        if testVector[featureIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featureNames, testVector)
            else:
            	classLabel = secondDict[key]
    
    return classLabel


In [256]:
classLabel = classify(myDecisionTree, featureNames, ['o', 'x', 'o', 'b', 'o', 'x', 'o', 'x', 'x'] )
print classLabel

#predict(myDecisionTree, featureNames, test_dataset):

AttributeError: 'str' object has no attribute 'keys'

### 优化（可选）
在上面这些步骤完成后，你可以优化 create_decision_tree 函数以防止过拟合

- 对决策树进行剪枝
- 也推荐两个更简单又十分有效的办法
    - 设置树的最大深度 max_depth
    - 设置每个叶节点的最小 samples 数
    - 这里可以参考 [decision tree in scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) 中的参数设置以及其原理

## 评估

下面的数据可在你的 predict 文件提交至 Kaggle 后获得。

- Kaggle 昵称：
- 模型目前 Public Leaderboard 得分：
- 排名：

### 反思
请对你的模型进行一定的分析，说出你模型的不足之处，或者可以提高的地方。

回答：